In [ ]:
%pip install --upgrade  langchain langchain-google-genai "langchain[docarray]" faiss-cpu pypdf

In [ ]:
%pip install -U google-ai-generativelanguage==0.6.15

In [ ]:
%pip install -U langchain-community

In [ ]:
import os
import requests
from PIL import Image

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display,Markdown

In [ ]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain.schema.document import Document
from langchain_community.document_loaders import TextLoader

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
%pip install -q -U google-generativeai langchain

In [ ]:
%pip install -q -U google-generativeai
import google.generativeai as genai

In [ ]:
!pip install faiss-cpu
import faiss


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
def load_model(model_name):
  if model_name=='gemini-2.5-flash':
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
  else:
    llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash-image-preview")

  return llm

In [ ]:
model_text=load_model("gemini-2.5-flash")

In [ ]:
# Configure your API key
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

In [ ]:
!pip show google-generativeai

In [ ]:
model_text(
    [
        HumanMessage(content="Answer with simple 'yes' or 'no'.Question: Is banana a Fruit?")
    ]
).content

In [ ]:
def get_image(url,filename,extension):
  content = requests.get(url).content
  with open(f'/content/{filename}.{extension}','wb') as f:
    f.write(content)
    image = Image.open(f"/content/{filename}.{extension}")
    image.show()
    return image

In [ ]:
image=get_image("https://cdn.mos.cms.futurecdn.net/yDn3ZSXu9eSBxmXQDZ4PCF.jpg","i-phone","jpg")

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
vision_model = load_model("gemini-2.5-flash-image-preview")

In [ ]:
message= HumanMessage(
    content=[
        {
            "type": "image_url", "image_url": image
        }
    ]
)

In [ ]:
loader = TextLoader("/content/iphone_details.txt")
print(loader.load()[0].page_content)

In [ ]:
text=loader.load()[0].page_content

In [ ]:
def get_text_chunks_langchain(text):
  text_splitter = CharacterTextSplitter(chunk_size=20, chunk_overlap=10)
  docs= [Document(page_content=x) for x in text_splitter.split_text(text)]
  return docs

In [ ]:
docs = get_text_chunks_langchain(text)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
retriever.invoke("Technical Specifications (sample)")

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
llm_vision=load_model("gemini-2.5-flash-image-preview")

In [ ]:
llm_text = load_model("gemini_pro")

In [ ]:
template = """
'''
{context}
'''
{query}

provide information
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context":retriever,"query":RunnablePassthrough()}
    |prompt
    |llm_text
    |StrOutputParser()
)

In [ ]:
result = rag_chain.invoke("can you give camera details?")

In [ ]:
display(Markdown(result))